In [ ]:
import os
import sys
PACKAGE_PARENT = '..'
sys.path.append(PACKAGE_PARENT)

import pandas as pd
from MGraphEmbedding.GraphConstruct import GraphConstruct
from matplotlib import pyplot as plt
import networkx as nx
#import nxmetis

In [ ]:
new_xukai = pd.read_pickle('../data/wd_nsr.pickle')['nsrdzdah']

In [ ]:

container = GraphConstruct()

Positive_namelist = ['../data/zzszyfp_xf.pickle', '../data/zzszyfp_gf.pickle']
Unlabeled_namelist = ['../data/zzszyfp_xf_unlabeled.pickle', '../data/zzszyfp_gf_unlabeled.pickle']

edges = container.get_all_edges_from_dataframe(container.merge_pickle(Positive_namelist + Unlabeled_namelist))
print('total edes num {}'.format(len(edges.drop_duplicates())))
G = container.construct_network(edges.values)

In [ ]:
# 为网络的节点添加属性
for node in G.nodes():
    G.add_node(node, xukai=False)
    
for node in new_xukai:
    G.add_node(node, xukai=True)
print(G.number_of_nodes())

# get degree of network

degree =  nx.degree_histogram(G)          #返回图中所有节点的度分布序列
x = range(len(degree))                             #生成x轴序列，从1到最大度
y = [z / float(sum(degree)) for z in degree]  
#将频次转换为频率，这用到Python的一个小技巧：列表内涵，Python的确很方便：）
plt.loglog(x,y)           #在双对数坐标轴上绘制度分布曲线  
plt.show()                                               

In [ ]:
# in degree 其实没有入度和出度，因为是无向图
import numpy as np
in_degrees = list(G.degree())
in_degree = [value[1] for value in in_degrees]
# max degree of the network
in_degree_num = {}
for i in in_degree:
    if i in in_degree_num:
        in_degree_num[i] += 1
    else:
        in_degree_num[i] = 1    
x = sorted(in_degree_num)
y = [in_degree_num[k] for k in x]
y = np.array(y) / G.number_of_nodes()
save = pd.DataFrame({'x':x, 'y':y})
save.to_pickle('./Figure/sh.pickle')
plt.xlabel('Degree')
plt.ylabel('Count')
plt.loglog(x, y)
plt.savefig('./Figure/shanghai.pdf')
# 度分布呈现幂率分布特性，典型的无标度网络
plt.show()
sum_degree = 0
for i in range(len(x)):
    sum_degree = sum_degree + x[i] * y[i]
average_degree = sum_degree / len(G.nodes())
print('average degree', average_degree)

In [ ]:
# 集聚系数 平均集聚系数
#随机网络的平均聚集系数Crand为：0.0003463，C/Crand=78.689，所以C>>Crand，该社交网络的平均聚集系数这一特征也符合小世界网络的特征。

In [ ]:
clusters = nx.clustering(G)
avg_clusters = nx.average_clustering(G)
avg_clusters

In [ ]:
# cliques   团，小圈子
cliques = nx.graph_number_of_cliques(G)
cliques

In [ ]:
# density
density = nx.density(G)
density
# 密度非常低表示边的连接比较稀疏

In [ ]:
# subgraphs

In [ ]:
subgraphs_nodes = nx.connected_components(G)
subgraphs = []
for c in sorted(subgraphs_nodes, key=len,reverse=True):
    print(len(c))
    subgraphs.append(G.subgraph(c))
print('number of subgraphs {}'.format(len(subgraphs)))

In [ ]:
# 直径
# 直径表示交易的长度
# diameter = nx.diameter(subgraphs[0])
# diameter

In [ ]:
# 中心度
# 1， 点中心度， 2， 紧密中心度， 3， 介数中心度， 4， 特征向量中心度


In [ ]:
# graph partitioning with Metis
import nxmetis
gg = subgraphs[2]
nx.draw_networkx(gg, with_labels=False)
sub_gg = nxmetis.partition(gg, 2, recursive=True)
sub_gg[0]

In [ ]:
ss = subgraphs[2].subgraph(sub_gg[1][1])
nx.draw_networkx(ss, with_labels=False)

In [ ]:
import community
count = 1
size = float(len(set(partitions.values()))) + 1
partitions = community.best_partition(subgraphs[2])
for com in set(partitions.values()):
    count += 1
    list_nodes = [nodes for nodes in partitions.keys() if partitions[nodes] == com]
    nx.draw_networkx(subgraphs[2].subgraph(list_nodes), pos = nx.spring_layout(subgraphs[2]), with_labels=False, node_color=str(1 - count / size))

In [ ]:
import community
def get_subgraphs(G):
    subgraphs_nodes = nx.connected_components(G)
    subgraphs = []
    for c in sorted(subgraphs_nodes, key=len,reverse=True):
        subgraphs.append(G.subgraph(c))
    new_subgraphs = []
    new_subgraphs = new_subgraphs + subgraphs[1:]
    partitions = community.best_partition(subgraphs[0])
    for com in set(partitions.values()):
        list_nodes = [nodes for nodes in partitions.keys() if partitions[nodes] == com]
        new_subgraphs.append(nx.Graph(subgraphs[0].subgraph(list_nodes).edges()))
    new_subgraphs = sorted(new_subgraphs, key=len, reverse=True)
    print(len(new_subgraphs))
    for i, graph in enumerate(new_subgraphs) :
        if len(graph.nodes) > 1000:
            print('{} graph information'.format(i))
            print('nodes number {}'.format(len(graph.nodes())))
            print('edges number {}\n\n'.format(len(graph.edges())))
            nx.write_gexf(graph, './Figure/ShangHai/' +str(i) + '.gexf')
    return new_subgraphs

In [ ]:
a = get_subgraphs(G)

In [ ]:
def get_uniform_node_number_subgraphs(G):
    import community
    subgraphs_nodes = nx.connected_components(G)
    subgraphs = []
    for c in sorted(subgraphs_nodes, key=len,reverse=True):
        ap_graph = nx.Graph()
        ap_graph.add_nodes_from(c)
        ap_graph.add_edges_from(G.subgraph(c).edges())
        subgraphs.append(ap_graph)
    new_subgraphs = []
    new_subgraphs = new_subgraphs + subgraphs[1:]
    partitions = community.best_partition(subgraphs[0])
    for com in set(partitions.values()):
        list_nodes = [nodes for nodes in partitions.keys() if partitions[nodes] == com]
        ap_graph = nx.Graph()
        ap_graph.add_nodes_from(list_nodes)
        ap_graph.add_edges_from(subgraphs[0].subgraph(list_nodes).edges())
        new_subgraphs.append(ap_graph)
    new_subgraphs = sorted(new_subgraphs, key=len)


    print('total subgraphs ', len(new_subgraphs))
    result_subgraphs = []
    insert_subgraphs = []
    total_nodes = 0
    index = 0
    for g in new_subgraphs:
        if index == 0:
            insert_subgraphs.append(g)
            total_nodes += g.number_of_nodes()
            index += 1
            continue
        if total_nodes > 1000:
            result_subgraphs.append(conn_graphs(insert_subgraphs))
            total_nodes = 0
            insert_subgraphs = []
            index += 1
            insert_subgraphs.append(g)
        else:
            insert_subgraphs.append(g)
            total_nodes += g.number_of_nodes()
            index += 1
    if total_nodes > 0:
        result_subgraphs.append(conn_graphs(insert_subgraphs))
    result_subgraphs = sorted(result_subgraphs, key=len)
    total_num = 0
    for gr in result_subgraphs:
        total_num += gr.number_of_nodes()
    print('result nodes, ', total_num)
    print('final get subgraphs ', len(result_subgraphs))
    return result_subgraphs

In [ ]:
b = get_uniform_node_number_subgraphs(G)